# Quantization of Image Classification Models

This tutorial demostrates how to apply INT8 quantization to Image Classification model using [Post-training Optimization Tool API](../../compression/api/README.md). The Mobilenet V2 model trained on the ImageNet-tiny dataset from Torchvision is used as an example. The code of this tutorial is designed to be extandable to custom model and dataset. It consists of the following steps:
- Install OpenVINO and required tools and packages using PIP manager
- Prepare the model for quantization
- Define data loading and accuracy validation functionality
- Run optimization pipeline
- Compare accuracy of the original and quantized models
- Compare performance of the original and quantized models

In [ ]:
import os
import sys
from pathlib import Path

import numpy as np
import torch
from addict import Dict

sys.path.append('../utils')

# Prepare dataset

 In this tutorial CIFAR10 will be used for quantization. It is downloaded inside DataLoader.

In [ ]:
# Set the data and model directories
DATA_DIR = 'data'
MODEL_DIR = 'model'

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

## Prepare the Model
Model preparation stage has the following steps:
- Download PyTorch model from Torchvision repository
- Convert it to ONNX format
- Run OpenVINO Model Optimizer tool to convert ONNX to OpenVINO Intermediate Representation (IR)



In [ ]:
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_mobilenetv2_x1_0", pretrained=True)
model.eval()

dummy_input = torch.randn(1, 3, 32, 32)

onnx_model_path = Path(MODEL_DIR) / 'mobilenet_v2.onnx'
ir_model_xml = onnx_model_path.with_suffix('.xml')
ir_model_bin = onnx_model_path.with_suffix('.bin')

torch.onnx.export(model, dummy_input, onnx_model_path, verbose=True)

# Run OpenVINO Model Optimization tool to convert ONNX to OpenVINO IR
!mo --framework=onnx --data_type=FP16 --input_shape=[1,3,32,32] -m $onnx_model_path  --output_dir $MODEL_DIR

## Define Data Loader
At this step the `DataLoader` interface from POT API is implemented.

In [ ]:
from torchvision import transforms
from torchvision.datasets import CIFAR10

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = CIFAR10(root=DATA_DIR, train=False, transform=transform, download=True)

In [2]:
import os
from compression.api import Metric, DataLoader


class CifarDataLoader(DataLoader):

    def __init__(self, config):
        if not isinstance(config, Dict):
            config = Dict(config)
        super().__init__(config)
        self.indexes, self.pictures, self.labels = self.load_data(dataset)
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        if index >= len(self):
            raise IndexError

        return (self.indexes[index], self.labels[index]), self.pictures[index].unsqueeze(0)

    def load_data(self, dataset):
        pictures, labels, indexes = [], [], []
        
        for idx, sample in enumerate(dataset):
            pictures.append(sample[0])
            labels.append(sample[1])
            indexes.append(idx)

        return indexes, pictures, labels

ModuleNotFoundError: No module named 'torchvision'

## Define Accuracy Metric Calculation
At this step the `Metric` interface for accuracy Top-1 metric is implemented. It is used for validating accuracy of quantized model.

In [3]:
# Custom implementation of classification accuracy metric.

class Accuracy(Metric):

    # Required methods
    def __init__(self, top_k=1):
        super().__init__()
        self._top_k = top_k
        self._name = 'accuracy@top{}'.format(self._top_k)
        self._matches = []

    @property
    def value(self):
        """ Returns accuracy metric value for the last model output. """
        return {self._name: self._matches[-1]}

    @property
    def avg_value(self):
        """ Returns accuracy metric value for all model outputs. """
        return {self._name: np.ravel(self._matches).mean()}

    def update(self, output, target):
        """ Updates prediction matches.
        :param output: model output
        :param target: annotations
        """
        if len(output) > 1:
            raise Exception('The accuracy metric cannot be calculated '
                            'for a model with multiple outputs')
        if isinstance(target, dict):
            target = list(target.values())
        predictions = np.argsort(output[0], axis=1)[:, -self._top_k:]
        match = [float(t in predictions[i]) for i, t in enumerate(target)]

        self._matches.append(match)

    def reset(self):
        """ Resets collected matches """
        self._matches = []

    def get_attributes(self):
        """
        Returns a dictionary of metric attributes {metric_name: {attribute_name: value}}.
        Required attributes: 'direction': 'higher-better' or 'higher-worse'
                             'type': metric type
        """
        return {self._name: {'direction': 'higher-better',
                             'type': 'accuracy'}}

NameError: name 'Metric' is not defined

## Run Quantization Pipeline and compare the accuracy of the original and quantized models
Here we define a configuration for our quantization pipeline and run it. 

**Note**: we use built-in `IEEngine` implementation of the `Engine` interface from the POT API for model inference. `IEEngine` is built on top of OpenVINO Python* API for inference and provides basic functionality for inference of simple models, ImageNet pre-trained models. If you have a more complicated inference flow for your model/models you should create your own implementation of `Engine` interface, for example by inheriting from `IEEngine` and extending it.

In [ ]:
from compression.graph import load_model, save_model
from compression.graph.model_utils import compress_model_weights
from compression.engines.ie_engine import IEEngine
from compression.pipeline.initializer import create_pipeline

model_config = Dict({
    'model_name': 'mobilenet_v2',
    'model': ir_model_xml,
    'weights': ir_model_bin
})
engine_config = Dict({
    'device': 'CPU',
    'stat_requests_number': 2,
    'eval_requests_number': 2
})
dataset_config = {
    'data_source': DATA_DIR
}
algorithms = [
    {
        'name': 'DefaultQuantization',
        'params': {
            'target_device': 'CPU',
            'preset': 'performance',
            'stat_subset_size': 300
        }
    }
]

# Steps 1-7: Model optimization
# Step 1: Load the model.
model = load_model(model_config)

# Step 2: Initialize the data loader.
data_loader = CifarDataLoader(dataset_config)

# Step 3 (Optional. Required for AccuracyAwareQuantization): Initialize the metric.
metric = Accuracy(top_k=1)

# Step 4: Initialize the engine for metric calculation and statistics collection.
engine = IEEngine(engine_config, data_loader, metric)

# Step 5: Create a pipeline of compression algorithms.
pipeline = create_pipeline(algorithms, engine)

# Step 6: Execute the pipeline.
compressed_model = pipeline.run(model)

# Step 7 (Optional): Compress model weights quantized precision
#                    in order to reduce the size of final .bin file.
compress_model_weights(compressed_model)

# Step 8: Save the compressed model to the desired path.
compressed_model_paths = save_model(model=compressed_model, save_path=MODEL_DIR, model_name="quantized_mobilenet_v2"
)
compressed_model_xml = compressed_model_paths[0]["model"]

# Step 9: Compare accuracy of the original and quantized models.
metric_results = pipeline.evaluate(model)
if metric_results:
    for name, value in metric_results.items():
        print('Accuracy of the original model: {: <27s}: {}'.format(name, value))

metric_results = pipeline.evaluate(compressed_model)
if metric_results:
    for name, value in metric_results.items():
        print('Accuracy of the optimized model: {: <27s}: {}'.format(name, value))

## Compare Performance of the Original and Quantized Models

Finally, we will measure the inference performance of the FP32 and INT8 models. To do this, we use [Benchmark Tool](https://docs.openvinotoolkit.org/latest/openvino_inference_engine_tools_benchmark_tool_README.html) - OpenVINO's inference performance measurement tool.

NOTE: For more accurate performance, we recommended running benchmark_app in a terminal/command prompt after closing other applications. Run benchmark_app -m model.xml -d CPU to benchmark async inference on CPU for one minute. Change CPU to GPU to benchmark on GPU. Run benchmark_app --help to see an overview of all command line options.


In [ ]:
# Inference FP16 model (IR)
!benchmark_app -m $ir_model_xml -d CPU -api async

In [ ]:
# Inference INT8 model (IR)
!benchmark_app -m $compressed_model_xml -d CPU -api async